# Compact pathway creator function
Defines all pathways and returns a model with the selected pathway added.  
The function can be called alone or get assigned to a new variable, but it always alters the handed model. In order to not touch manipulate the existing model work within _with model:_ or create a copy of your model with _model.copy()_ before handing that copy to this function.
### Function
**create_pathway**(_model,path_selector_)  

model ... needs to be handed a model, since it doesnt import it itself to save time when function if called multiple times  
path_selector ... determines which path is added  
Import the function into your script with the following code, thic function script has to be in the same folder.  
   
_%run -i create_pathway.ipynb_
### Pathway legend
0 -> no additional reactions  
1 -> beta alanine pathway  
2 -> transferase malonyl-CoA pathway  
3 -> oxidoreductive malonyl-CoA pathway  
4 -> lactate pathway  

In [2]:
path_name = ['no additional reactions','beta alanine pathway','trans malonyl-CoA pathway','red malonyl-CoA pathway','lactate pathway']

## General
Borodina et al. describe 4 different pathways to get to 3-hydroxy propionic acid with *Komatogaella phaffii*, where most of them go through malonic semialdehyde (Borodina 2014). This metabolite can serve as a precursor for both, malonic acid and 3-hydroxy propionic acid and therfore these pathways serve as a starting point for this cell factory project. The reactions, enzyms and corresponding metabolites were explored using the Kyoto Encyclopedia of Genes and Genomes KEGG (Kanehisa 2000, Kanehisa 2019, Kanehisa 2021). The described routes and the included are:
- Beta-Alanine
- Malonyl-CoA
- Lactate
- Glycerol
  
The pathways are established in the model by inserting one or several new enzyms and deleting competing ones that draw from the pathway.

The glycerol route was dropped because it needs B12 as a cofactor, which is not natural in *K. phaffii*.  

### Beta alanine pathway
For this pathway two steps are necessary, first beta-alanine is changed to malonate semialdehyde by transferring its amino group onto alpha-ketoglutaric acid. For this reaction beta-alanine-oxoglutarate aminotransferase is need, which is already implemented in the used model.  
For the second step the enzym malonate-semialdehyde dehydrogenase is used, which hydrolises the malonate semialdehyde to malonic acid and uses NADP+ as a cofactor.

In [3]:
# malonate-semialdehyde dehydrogenase
mal_semal_dehydr = Reaction('MSADH')
mal_semal_dehydr.name = 'malonate-semialdehyde dehydrogenase'
# 3-Oxopropanoate + NADP+ + H2O <=> Malonate + NADPH + H+
malonic_acid_c = Metabolite(id='mln_c', compartment='c')
malonic_acid_c.name = 'Malonic acid'
malonic_acid_c.formula = 'C3H2O4'

mal_semal_dehydr.add_metabolites({model.metabolites.msa_c: -1,  # = 3-oxopropanoate
                                 model.metabolites.nad_c: -1,
                                 model.metabolites.h2o_c: -1,
                                 model.metabolites.nadh_c: 1,
                                 model.metabolites.h_c: 1,
                                 malonic_acid_c: 1})

In [4]:
# Set up pathway
beta_al_path = [mal_semal_dehydr]

### Malonyl-CoA route
For malonyl-coa as a precursor, two pathways were identified and both are implemented. The first one directly switches the malonic acid group with an acetate molecule to create malonic acid and acetyl-coa. It uses Acetyl-CoA:malonate CoA-transferase and is further referred to as the Malonly-CoA transferase pathway.  
  
The second option is a two step process, where malonyl-coa is split into malonate semialdehyde and co-enzym A while using NADPH through 3-oxopropanoate:NADP+ oxidoreductase (CoA-malonylating). Afterwards the same enzym as in the beta-alanine pathway is used to produce malonic acid. In order to force the model to work through this pathway, the enzym beta-alanine-oxoglutarate aminotransferase has to be shut down, which is described in the beta-alanine pathway as well. Because of the nature of the reactions it is called the Malony-Coa oxidored pathway.

In [ ]:
# malonyl_option = 'A'
# valid options A or B, see below

In [ ]:
# Option A
# Acetyl-CoA:malonate CoA-transferase
malcoa_trans = Reaction('MCOATR')
malcoa_trans.name = '# Acetyl-CoA:malonate CoA-transferase'
#Acetate + Malonyl-CoA <=> Acetyl-CoA + Malonate

malcoa_trans.add_metabolites({model.metabolites.ac_c: -1,
                                 model.metabolites.malcoa_c: -1,
                                 model.metabolites.accoa_c: 1,
                                 malonic_acid_c: 1})

In [5]:
# Option B
# 3-oxopropanoate:NADP+ oxidoreductase (CoA-malonylating)
malcoa_red = Reaction('MCOARED')
malcoa_red.name = '3-oxopropanoate:NADP+ oxidoreductase (CoA-malonylating)'
# 3-Oxopropanoate + CoA + NADP+ <=> Malonyl-CoA + NADPH + H+

malcoa_red.add_metabolites({model.metabolites.msa_c: 1,
                                 model.metabolites.nadp_c: 1,
                                 model.metabolites.coa_c: 1,
                                 model.metabolites.malcoa_c: -1,
                                 model.metabolites.nadph_c: -1,
                                 model.metabolites.h_c: -1})

In [6]:
# set up pathway
# if malonyl_option == 'A':  # Option A
trans_malcoa_path = [malcoa_trans]
# elif malonyl_option =='B': # Option B
red_malcoa_path = [malcoa_red, mal_semal_dehydr]


### Lactate route
In order to produce malonic acid through lactate several enzyms have to be included. Initially lactate is transfered onto co-enzym A, followed by several hydratase steps leading up to 3Hydroxypropyionyl-CoA. From here it is fragmented into 3HP and CoA, where 3HP will be transformed to malonate semialdehyde and finally to malonic acid. The included enzyms are consecutively propionate CoA-transferase, lactoyl-CoA dehydratase, 3-Hydroxypropionyl-CoA hydro-lyase, 3-hydroxyisobutyryl-CoA hydrolase, 3-hydroxypropanoate:NAD+ oxidoreductase and finally and 3-hydroxypropionyl-CoA:NADP+ oxidoreductase.

In [7]:
# propionate CoA-transferase
propCoA_trans = Reaction('PRCOATR')
propCoA_trans.name = 'Propionate CoA-transferase'
# (S)-Lactate + Acetyl-CoA <=> Lactoyl-CoA + Acetate
lacCoA = Metabolite(id='lact_coa', compartment='c')
lacCoA.name = 'Lactoyl-CoA'
lacCoA.formula = 'C24H40N7O18P3S'

propCoA_trans.add_metabolites({model.metabolites.lac_L_c: -1,
                                 model.metabolites.accoa_c: -1,
                                 lacCoA: 1,
                                 model.metabolites.ac_c: 1})

In [8]:
# Lactoyl-CoA hydrolyase
lacCoA_hyd = Reaction('LACOAHYD')
lacCoA_hyd.name = 'Lactoyl-CoA hydrolyase'
# Lactoyl-CoA <=> Propenoyl-CoA (Acrylyl-CoA) + H2O
propCoA = Metabolite(id='prop_coa', compartment='c')
propCoA.name = 'Propenoyl-CoA'
propCoA.formula = 'C24H38N7O17P3S'

lacCoA_hyd.add_metabolites({lacCoA: -1,
                                 propCoA: 1,
                                 model.metabolites.h2o_c: 1})

In [9]:
# 3-Hydroxypropionyl-CoA hydrolyase
hydpropCoA_hyd = Reaction('3HPCOAHYD')
hydpropCoA_hyd.name = '3-Hydroxypropionyl-CoA hydrolyase'
# Propenoyl-CoA + H2O <=> 3-Hydroxypropionyl-CoA
hydpropCoA = Metabolite(id='hydprop_coa', compartment='c')
hydpropCoA.name = '3-Hydroxypropionyl-CoA'
hydpropCoA.formula = 'C24H40N7O18P3S'

hydpropCoA_hyd.add_metabolites({propCoA: -1,
                                 model.metabolites.h2o_c: -1,
                                 hydpropCoA: 1})

In [10]:
# Dissosication
hydpropCoA_dis = Reaction('3HPCOADIS')
hydpropCoA_dis.name = '3-Hydroxypropionyl-CoA Dissociation (not systematic)'
# 3-Hydroxypropionyl-CoA + H2O <=> 3-Hydroxypropanoate + CoA
hydprop = Metabolite(id='hydprop', compartment='c')
hydprop.name = '3-Hydroxypropanoate'
hydprop.formula = 'C3H6O3'

hydpropCoA_dis.add_metabolites({hydpropCoA: -1,
                                 model.metabolites.h2o_c: -1,
                                 hydprop: 1,
                                 model.metabolites.coa_c: 1})

In [11]:
# 3-hydroxypropanoate:NAD+ oxidoreductase
hydprop_red = Reaction('3HPRED')
hydprop_red.name = '3-hydroxypropanoate:NAD+ oxidoreductase'
# 3-Hydroxypropanoate + NAD+ <=> 3-Oxopropanoate + NADH + H+

hydprop_red.add_metabolites({hydprop: -1,
                                 model.metabolites.nad_c: -1,
                                 model.metabolites.msa_c: 1,
                                 model.metabolites.nadh_c: 1,
                                 model.metabolites.h_c: 1})

In [12]:
lactate_path = [propCoA_trans, lacCoA_hyd, hydpropCoA_hyd, hydpropCoA_dis, hydprop_red, mal_semal_dehydr]

### Transport reactions for malonic acid
For a more realistic model, the transport reaction from the cytosol, where all the reactions in this case take part, to the extracellular space, as well as the removeal reaction Ex_mln are included.

In [13]:
# Transport reaction cytosol -> extracellular
mln_ctoe = Reaction('mln_c-e')
mln_ctoe.name = 'Malonic acid trans c -> e'
malonic_acid_e = Metabolite(id='mln_e', compartment='e')
malonic_acid_e.name = 'Malonic acid'
malonic_acid_e.formula = 'C3H2O4'
mln_ctoe.add_metabolites({malonic_acid_c: -1,
                           malonic_acid_e: 1})
# Removal reaction -> alernativ implementation as boundary within the create_pathway function
# Ex_mln = Reaction('Ex_mln')
#Ex_mln.name = 'Malonic acid ex/inport'
#Ex_mln.add_metabolites({malonic_acid_e: -1})
# set up transport path
transport = [mln_ctoe]
#model.add_boundary(model.metabolites.malonic_acid_c, type='demand')

### Adding selected pathway to model

In [14]:
pathways = [[],beta_al_path,trans_malcoa_path,red_malcoa_path,lactate_path]

In [ ]:
def create_pathway(model, path_selector=0):
    model.add_reactions(pathways[path_selector])
    model.add_reactions(transport)
    model.add_boundary(model.metabolites.get_by_id('mln_e'), type='demand', reaction_id = 'Ex_mln')
    if path_selector == 4:  # force organism in the malonic acid pathway as indicated in the lactate route
        model.reactions.APAT2r.upper_bound = 0
        model.reactions.L_LACD2cm.upper_bound = 0
    elif path_selector == 3: #and malonyl_option =='B' to force the malonic semialdehyde (precursor) production through malonyl-CoA instead of beta alnine path
        model.reactions.APAT2r.upper_bound = 0

    print('Added', path_name[path_selector])
    return model